In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%cd ../input/dog-breed-identification/

In [ ]:
!ls

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import time
import copy
import warnings
import random
import numpy as np
import pandas as pd
import torchvision
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import albumentations as albu
import matplotlib.image as mpi

from albumentations import (HorizontalFlip,VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise,RandomRotate90,Transpose,RandomBrightnessContrast,RandomCrop)
from albumentations.pytorch import ToTensor
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from torchvision import models
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import lr_scheduler
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
seed = 69
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
sub = pd.read_csv('sample_submission.csv')

In [ ]:
sub.keys()

In [ ]:
NUM_CLASSES = 120
labels = pd.read_csv("labels.csv")
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)].reset_index(drop=True)

In [ ]:
df1 = labels['breed']
df2 = labels["id"]
df1 = pd.get_dummies(df1)
df = pd.concat([df2,df1], axis=1)

In [ ]:
df_train,df_val = train_test_split(df,test_size=0.2,random_state=42)

In [ ]:
class DogDataset(Dataset):

  def __init__(self,df,root,phase):
    self.df = df
    self.length = df.shape[0]
    self.root = root
    if phase=="train":
        self.transforms = albu.Compose([
            albu.SmallestMaxSize(256),
            albu.RandomCrop(256,256),
            albu.HorizontalFlip(p=0.5),
            albu.Cutout(),
            albu.RGBShift(),
            albu.Rotate(limit=(-90,90)),
            # albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
        ])
    elif phase=="val":
        self.transforms = albu.Compose([
            albu.Resize(256,256),
            # albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
        ])

  def __getitem__(self,index):
    label = self.df.iloc[index,1:]
    label = label.to_numpy()
    image_id = self.df.iloc[index,0]
    path = os.path.join(self.root,str(image_id) + ".jpg")
    img = plt.imread(path)
    img = self.transforms(image=np.array(img))
    img = img['image']
    img = np.transpose(img,(2,0,1)).astype(np.float32)
    img = torch.tensor(img, dtype = torch.float)
    label = np.argmax(label)
    return img,label
  
  def __len__(self):
    return self.length 
  
  def label_name(self,label):
    breeds = self.df.columns.values
    breeds = breeds[1:]
    idx = np.argmax(label)
    return breeds[idx]

In [ ]:
def train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    dataset_sizes = {'train': len(dataloaders['train'].dataset), 
                     'val': len(dataloaders['val'].dataset)}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    number_of_iter = 0
    acc_train = []
    acc_val = []
    loss_train = []
    loss_val = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            current_loss = 0.0
            current_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]
            if phase=="train":
                acc_train.append(epoch_acc)
                loss_train.append(epoch_loss)
            else:
                acc_val.append(epoch_acc)
                loss_val.append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    
    
    return model,acc_val,acc_train,loss_train,loss_val

In [ ]:
traindata = DogDataset(df_train,root = "train", phase="train")
valdata = DogDataset(df_val,root = "train", phase="val")
trainloader = DataLoader(traindata,batch_size = 24,num_workers=0)
valloader = DataLoader(valdata,batch_size = 24,num_workers=0)

In [ ]:
dataiter = iter(trainloader)
image,label = dataiter.next()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def show_img(img):
    plt.figure(figsize=(18,15))
    img = img / 2 + 0.5  
    npimg = img.numpy()
    npimg = np.clip(npimg, 0., 1.)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

show_img(torchvision.utils.make_grid(image))

## Resnet152

In [ ]:
__all__ = ['ResNet50', 'ResNet101','ResNet152']

def Conv1(in_planes, places, stride=2):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_planes,out_channels=places,kernel_size=7,stride=stride,padding=3, bias=False),
        nn.BatchNorm2d(places),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

class Bottleneck(nn.Module):
    def __init__(self,in_places,places, stride=1,downsampling=False, expansion = 4):
        super(Bottleneck,self).__init__()
        self.expansion = expansion
        self.downsampling = downsampling

        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_places,out_channels=places,kernel_size=1,stride=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(places),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=places, out_channels=places*self.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(places*self.expansion),
        )

        if self.downsampling:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_places, out_channels=places*self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(places*self.expansion)
            )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        residual = x
        out = self.bottleneck(x)

        if self.downsampling:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self,blocks, num_classes=120, expansion = 4):
        super(ResNet,self).__init__()
        self.expansion = expansion

        self.conv1 = Conv1(in_planes = 3, places= 64)

        self.layer1 = self.make_layer(in_places = 64, places= 64, block=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_places = 256,places=128, block=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_places=512,places=256, block=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_places=1024,places=512, block=blocks[3], stride=2)

        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(2048,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def make_layer(self, in_places, places, block, stride):
        layers = []
        layers.append(Bottleneck(in_places, places,stride, downsampling =True))
        for i in range(1, block):
            layers.append(Bottleneck(places*self.expansion, places))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
def ResNet50():
    return ResNet([3, 4, 6, 3])

def ResNet101():
    return ResNet([3, 4, 23, 3])

def ResNet152():
    return ResNet([3, 8, 36, 3])

In [ ]:
resnet152 = ResNet152()
fc_inputs = resnet152.fc.in_features
cs_net = nn.Sequential(
            nn.Linear(fc_inputs, NUM_CLASSES*12),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*12, NUM_CLASSES*6),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*6, NUM_CLASSES),
)
resnet152.fc = cs_net

In [ ]:
model = resnet152
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer,factor=0.33, mode="min", patience=2)
dataloaders = {"train":trainloader,"val":valloader}
start_time = time.time()
model_resnet, acc_val_resnet, acc_train_resnet, loss_train_resnet, loss_val_resnet = \
        train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25)

## Resnet50

In [ ]:
resnet50 = ResNet50()
fc_inputs = resnet50.fc.in_features
cs_net = nn.Sequential(
            nn.Linear(fc_inputs, NUM_CLASSES*12),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*12, NUM_CLASSES*6),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*6, NUM_CLASSES),
)
resnet50.fc = cs_net

In [ ]:
model = resnet50
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer, factor=0.33, mode="min", patience=2)
dataloaders = {"train":trainloader,"val":valloader}
model_resnet50, acc_val_resnet50, acc_train_resnet50, loss_train_resnet50, loss_val_resnet50 = \
        train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25)

## GoogleNet

In [ ]:
googlenet = models.googlenet(pretrained=True).to(device)
for param in googlenet.parameters():
    param.requires_grad=False
fc_inputs = googlenet.fc.in_features
googlenet.fc = nn.Linear(fc_inputs,NUM_CLASSES)

In [ ]:
model = googlenet
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer,factor=0.33, mode="min", patience=2)
model = model.to(device)
dataloaders = {"train":trainloader,"val":valloader}
num_epochs = 15
start_time = time.time()
model_googlenet, acc_val_googlenet, acc_train_googlenet, loss_train_googlenet, loss_val_googlenet = \
        train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=)

## DenseNet

In [ ]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module("norm1", nn.BatchNorm2d(num_input_features))
        self.add_module("relu1", nn.ReLU(inplace=True))
        self.add_module("conv1", nn.Conv2d(num_input_features, bn_size*growth_rate,
                                           kernel_size=1, stride=1, bias=False))
        self.add_module("norm2", nn.BatchNorm2d(bn_size*growth_rate))
        self.add_module("relu2", nn.ReLU(inplace=True))
        self.add_module("conv2", nn.Conv2d(bn_size*growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1, bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)

class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features+i*growth_rate, growth_rate, bn_size,
                                drop_rate)
            self.add_module("denselayer%d" % (i+1,), layer)

class _Transition(nn.Sequential):
    def __init__(self, num_input_feature, num_output_features):
        super(_Transition, self).__init__()
        self.add_module("norm", nn.BatchNorm2d(num_input_feature))
        self.add_module("relu", nn.ReLU(inplace=True))
        self.add_module("conv", nn.Conv2d(num_input_feature, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module("pool", nn.AvgPool2d(2, stride=2))
        
class DenseNet(nn.Module):
    "DenseNet-BC model"
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16), num_init_features=64,
                 bn_size=4, compression_rate=0.5, drop_rate=0, num_classes=1000):
        super(DenseNet, self).__init__()
        # first Conv2d
        self.features = nn.Sequential(OrderedDict([
            ("conv0", nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ("norm0", nn.BatchNorm2d(num_init_features)),
            ("relu0", nn.ReLU(inplace=True)),
            ("pool0", nn.MaxPool2d(3, stride=2, padding=1))
        ]))

        # DenseBlock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers, num_features, bn_size, growth_rate, drop_rate)
            self.features.add_module("denseblock%d" % (i + 1), block)
            num_features += num_layers*growth_rate
            if i != len(block_config) - 1:
                transition = _Transition(num_features, int(num_features*compression_rate))
                self.features.add_module("transition%d" % (i + 1), transition)
                num_features = int(num_features * compression_rate)

        # final bn+ReLU
        self.features.add_module("norm5", nn.BatchNorm2d(num_features))
        self.features.add_module("relu5", nn.ReLU(inplace=True))

        # classification layer
        self.classifier = nn.Linear(num_features, num_classes)

        # params initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.bias, 0)
                nn.init.constant_(m.weight, 1)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.avg_pool2d(features, 7, stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        return out
    
def densenet121(pretrained=False, **kwargs):
    """DenseNet121"""
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16),
                     **kwargs)

    if pretrained:
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')
        state_dict = model_zoo.load_url(model_urls['densenet121'])
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        model.load_state_dict(state_dict)
    return model

densenet = densenet121(pretrained=True)
classifier_inputs = densenet.classifier.in_features
cs_net = nn.Sequential(
            nn.Linear(classifier_inputs, NUM_CLASSES*12),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*12, NUM_CLASSES*6),
            nn.ReLU(inplace=True),
            nn.Linear(NUM_CLASSES*6, NUM_CLASSES),
)
densenet.classifier = cs_net

In [ ]:
model = densenet
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer,factor=0.33, mode="min", patience=2)
dataloaders = {"train":trainloader, "val":valloader}
start_time = time.time()
model_densenet, acc_val_densenet, acc_train_densenet, loss_train_densenet, loss_val_densenet = \
        train_model(dataloaders, model, criterion, optimizer, scheduler, num_epochs=25)

In [ ]:
epoch = []
for x in range(num_epochs):
    epoch.append(x)
plt.plot(epoch,loss_train,label = 'TrainLoss')
plt.plot(epoch,loss_val,label = 'ValLoss')
plt.legend()
plt.show()

In [ ]:
print(plt.style.available)

In [ ]:
epoch = list(range(1, 26))
plt.style.use('default')
plt.plot(epoch, acc_train_densenet, label='DenseNet')
plt.plot(epoch, acc_train_googlenet, label='GoogleNet')
plt.plot(epoch, acc_train_resnet, label='ResNet152')
plt.plot(epoch, acc_train_resnet50, label='ResNet50')
plt.ylabel('Training Set Accuracy')
plt.xlabel('Epoch')
plt.grid(linestyle='-.')
plt.legend(loc=(0.72, 0.1))
plt.show()

In [ ]:
epoch = list(range(1, 26))
plt.style.use('default')
plt.plot(epoch, acc_val_densenet, label='DenseNet')
plt.plot(epoch, acc_val_googlenet, label='GoogleNet')
plt.plot(epoch, acc_val_resnet, label='ResNet152')
plt.plot(epoch, acc_val_resnet50, label='ResNet50')
plt.ylabel('Validation Set Accuracy')
plt.xlabel('Epoch')
plt.grid(linestyle='-.')
plt.legend(loc=(0.72, 0.1))
plt.show()

In [ ]:
epoch = list(range(1, 26))
plt.plot(epoch, loss_train_densenet, label='DenseNet')
plt.plot(epoch, loss_train_googlenet, label='GoogleNet')
plt.plot(epoch, loss_train_resnet, label='ResNet152')
plt.plot(epoch, loss_train_resnet50, label='ResNet50')
plt.ylabel('Training Set Loss')
plt.xlabel('Epoch')
plt.grid(linestyle='-.')
plt.legend()
plt.show()

In [ ]:
epoch = list(range(1, 26))
plt.plot(epoch, loss_val_densenet, label='DenseNet')
plt.plot(epoch, loss_val_googlenet, label='GoogleNet')
plt.plot(epoch, loss_val_resnet, label='ResNet152')
plt.plot(epoch, loss_val_resnet50, label='ResNet50')
plt.ylabel('Validation Set Loss')
plt.xlabel('Epoch')
plt.grid(linestyle='-.')
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(),'res152.pth')

In [ ]:
output = pd.DataFrame(index=sub.index,columns = sub.keys())
output['id'] = sub['id']

In [ ]:
testdata = DogDataset(sub,root="test",phase='val')
testloader = DataLoader(testdata,batch_size=24)

In [ ]:
def test_submission(model):
    since = time.time()
    sub_output = []
    model.train(False)
    for data in testloader:
        inputs,labels = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        sub_output.append(outputs.data.cpu().numpy())
    sub_output = np.concatenate(sub_output)
    for idx,row in enumerate(sub_output.astype('float')):
        sub_output[idx] = np.exp(row)/np.sum(np.exp(row))
    output.loc[:,1:] = sub_output
    print()
    time_elapsed = time.time() - since
    print('Run complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
        

In [ ]:
%cd '/kaggle/input/dog-breed-identification'

In [ ]:
model = model.to(device)
test_submission(model)

In [ ]:
output.head()

In [ ]:
%cd '/kaggle/working'

In [ ]:
output.to_csv("dogs_idres152.csv", index=False)